# Description du projet
Le projet s'intéresse au prix de l'immobilier sur Paris. Est-on en mesure d'avoir une bonne prédiction sur la valeur mobilière d'un bien

# Lecture des jeux de données
Les jeux de données sont disponibles sur https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Fonction pour lire les donnnées en fonction du fichier
def ReadFile(nomFile, delimiter = '|'):
    # lecture du fichier excel
    df = pd.read_csv(nomFile, delimiter = delimiter, low_memory = False)
    print("taille du jeu de donnees :", df.shape)
    return df
    
    

In [3]:
# Fonction pour extraire les données à partir d'un numéro de département
def ExtractDepartement(df, numDep):
    df['code_departement'].astype(str)
    df['Validation'] = (df['code_departement'] == numDep )
    dfDep = df[df['Validation']==True]
    dfDep = dfDep.drop('Validation', axis=1)
    print("Departement : {0}".format(numDep))
    print("Taille du jeu de donnees", dfDep.shape)
    return dfDep       

In [4]:
df1 = ReadFile("../input/AvecCoordonneesGeo/full.csv", ',')

taille du jeu de donnees : (1429093, 40)


In [5]:
df2 = ReadFile("../input/AvecCoordonneesGeo/full2021.csv", ',')

taille du jeu de donnees : (4375223, 40)


## Concaténation des données

In [6]:
#Concaténation des deux jeux de données
df = pd.concat([df1,df2])
#,keys=['2022','2021'])
#, names = ['FileInput', 'RowId'])
print("taille suite à union :", df.shape)

taille suite à union : (5804316, 40)


## Réduction à un département

In [7]:
dfDep = ExtractDepartement(df,'75')
dfDepIni = dfDep
# Enlever le commentaire pour générer le jeu de données pour Power BI
dfDepIni.to_csv('../input/AvecCoordonneesGeo/dep752.csv')

Departement : 75
Taille du jeu de donnees (130696, 40)


# Nettoyage du jeu de données

## Nettoyage 

In [ ]:
#Suppression des lignes en doublon
dfDep.drop_duplicates(inplace=True)
dfDep.shape

In [ ]:
#Suppression des longitudes et latitudes null
dfDep.drop(dfDep[(dfDep['longitude'].isnull()) | (dfDep['latitude'].isnull())].index, inplace=True)
dfDep.shape

In [ ]:
#Suppression des données où la valeur foncière est null
dfDep.drop(dfDep[dfDep['valeur_fonciere'].isnull() ].index, inplace=True)
dfDep.shape


In [ ]:
#Suppression des valeurs foncières < 50KE et >3000KE
dfDep.drop(dfDep[dfDep['valeur_fonciere']<50000 ].index, inplace=True)
dfDep.drop(dfDep[dfDep['valeur_fonciere']>3000000 ].index, inplace=True)
dfDep.shape

In [ ]:
#Suppression des variables qui sont nulles pour 80% des valeurs
listVariables = dfDep.isnull().sum() > (dfDep.shape[0]*0.8)
listResultatsVarDrop = []
for colname, serie in listVariables.iteritems():
    if(serie == True):
            listResultatsVarDrop.append(colname)
listResultatsVarDrop
dfDep.drop(listResultatsVarDrop, inplace=True, axis=1)
dfDep.shape

In [ ]:
#Conversion des objets en string
dfDep['adresse_nom_voie'] = dfDep['adresse_nom_voie'].astype("string")
dfDep['adresse_numero'] = dfDep['adresse_numero'].astype("string")
dfDep['nom_commune'] = dfDep['nom_commune'].astype("string")
dfDep['adresse_complete']=dfDep['adresse_numero']+' '+dfDep['adresse_nom_voie']+' , '+dfDep['nom_commune']

In [ ]:
#Suppression des données où le type de local est une dépendance
dfDep.drop(dfDep[dfDep['type_local']== 'Dépendance' ].index, inplace=True)
dfDep.shape

In [ ]:
dfDep.head(10)

In [ ]:
def AggregationSimilarData(df):
    
    # Construction d'un dictionnaire 
    # où la clé est la chaine de caractère qui permet d'indiquer que deux lignes sont similaires
    # où la valeur est l'index dans le dataframe initial
    dict_similarData = {}
    for index,series in df.iterrows():
        keyRow = str(series['date_mutation'])+'_'+str(series['valeur_fonciere'])+'_'+series['adresse_complete']
        if keyRow in dict_similarData:
            listIndexSimilaire = dict_similarData[keyRow]
            listIndexSimilaire.append(index)
        else:
            listKeyRow = list();
            listKeyRow.append(index)
            dict_similarData[keyRow] = listKeyRow
    
    #Suppression des valeurs dupliquées en prenant comme surface_reelle_bati le cumulé des surfaces
    listIndexASupprimer = []
    for cle,listIndex in dict_similarData.items():
        if(len(listIndex)>1):
            valSurfaceAgregee = df.at[listIndex[0],"surface_reelle_bati"]
            val = 1
            while (val != len(listIndex)):
                listIndexASupprimer.append(listIndex[val])
                valSurfaceAgregee += df.at[listIndex[val],"surface_reelle_bati"]
                val += 1
            df.at[listIndex[0],"surface_reelle_bati"] = valSurfaceAgregee
    #print(listIndexASupprimer)
    df.drop(listIndexASupprimer, inplace = True, axis = 0)
    print(df.shape)


## Gestion des doublons

In [ ]:
AggregationSimilarData(dfDep)

## Gestion des variables catégorielles
On regarde les valeurs uniques pour identifier les variables catégorielles

In [ ]:
for colname, serie in dfDep.iteritems():
    print(colname + " has " + str(serie.drop_duplicates().shape[0]) + " unique values.")

In [ ]:
dfDep["nature_mutation"] = pd.Categorical(dfDep["nature_mutation"], ordered=False)
dfDep["type_local"] = pd.Categorical(dfDep["type_local"], ordered=False)
dfDep["nombre_pieces_principales"] = pd.Categorical(dfDep["nombre_pieces_principales"], ordered=False)
dfDep["nom_commune"] = pd.Categorical(dfDep["nom_commune"], ordered=False)


In [ ]:
#Suppression des variables qui semblent inutiles
dfDep.drop(['code_departement', 'code_postal', 'adresse_code_voie', 'code_commune', 'id_parcelle','lot1_numero','lot2_numero', 'code_type_local'], inplace=True, axis=1)
dfDep.shape

## Typage des variables

In [ ]:
dfDep['date_mutation'] = pd.to_datetime(dfDep['date_mutation'], format='%Y/%m/%d')

In [ ]:
dfDep.info()

In [ ]:
#Conversion des objets en string
dfDep['id_mutation'] = dfDep['id_mutation'].astype("string")
dfDep.info()


# Exploration 

## Description univariée

### La variable de temps
On effectue du feature engineering

In [ ]:
dfDep['month']=dfDep["date_mutation"].apply(lambda x: x.month)
dfDep['day'] = dfDep["date_mutation"].apply(lambda x: x.day)
dfDep['year'] = dfDep["date_mutation"].apply(lambda x: x.year)
dfDep["month"] = pd.Categorical(dfDep["month"], ordered=True)
dfDep["day"] = pd.Categorical(dfDep["day"], ordered=True)
dfDep["year"]= pd.Categorical(dfDep["year"], ordered=True)

On va représenter les variables catégorielles à travers des tableaux de contingence ou des bars plots

In [ ]:
dfDep["year"].value_counts()

On observe qu'on a pratiquement le double de données entre 2021 et 2022 ce qui est normal car on a une vision partielle de 2022 avec des données jusqu'à Juin

In [ ]:
dfDep["month"].value_counts().sort_index().plot(kind="bar")
plt.title("Distribution des dates par mois")
plt.xlabel("Month")
plt.ylabel("Count")
plt.show()

On retrouve les données partielles. Par contre, il apparait difficile de faire des conclusions sur les mois hormi que le mois d'août semble relativement faible ce qui peut s'expliquer car les personnes sont en vacances

In [ ]:
dfDep["day"].value_counts().sort_index().plot(kind="bar")
plt.title("Distribution des dates par jour")
plt.xlabel("Day")
plt.ylabel("Count")
plt.show()

Il semblerait qu'il y ait plus de ventes en milieu et fin de mois

### La cible de notre modèle

In [ ]:
dfDep["valeur_fonciere"].describe()

In [ ]:
sns.boxplot(x=dfDep['valeur_fonciere'])

On a clairement des problèmes d'échelle avec des outliers à supprimer

In [ ]:
#Methode Remove outliers pour une loi biaisée
def removeOutliers(variable):
    print("avant ", dfDep.shape)
    Q1 = dfDep[variable].quantile(0.25)
    Q3 = dfDep[variable].quantile(0.75)
    IQR = Q3 - Q1
    dfDep.drop(dfDep[(dfDep[variable]<Q1 - 1.5*IQR) | (dfDep[variable]>Q3 + 1.5*IQR)].index, inplace=True)
    print("après ",dfDep.shape)

In [ ]:
removeOutliers('valeur_fonciere')

In [ ]:
#Methode Remove outliers par une loi normale
#Mean = dfDep['valeur_fonciere'].mean()
#StandardDeviation = dfDep['valeur_fonciere'].std()
#dfDep.drop(dfDep[(dfDep['valeur_fonciere']<Mean - 3*StandardDeviation) | (dfDep['valeur_fonciere']>Mean + 3*StandardDeviation)].index, inplace=True)
#dfDep.shape

In [ ]:
sns.boxplot(x=dfDep['valeur_fonciere'])
dfDep['valeur_fonciere'].describe()

### Les autres variables quantitatives

Regardons les variables abérrantes sur les max sur les m2 et mettons une valeur max à 500 m2

In [ ]:
#dfDep.drop(dfDep[dfDep['surface_reelle_bati']>500].index, inplace=True, axis=0)
#dfDep.drop(dfDep[dfDep['lot1_surface_carrez']>500].index, inplace=True, axis=0)
#dfDep.shape
#removeOutliers('surface_reelle_bati')
#dfDep.describe()

In [ ]:
#removeOutliers('lot1_surface_carrez')

In [ ]:
dfDep.describe()

In [ ]:
dfDep["prix m2"]=dfDep["valeur_fonciere"]/dfDep["surface_reelle_bati"]
sns.boxplot(x=dfDep['prix m2'])
dfDep["prix m2"].describe()

In [ ]:
removeOutliers("prix m2")

In [ ]:
dfDep["prix m2"]=dfDep["valeur_fonciere"]/dfDep["surface_reelle_bati"]
sns.boxplot(x=dfDep['prix m2'])
dfDep.describe()

In [ ]:
removeOutliers("lot1_surface_carrez")
sns.boxplot(x=dfDep['lot1_surface_carrez'])
dfDep.describe()

Les données sur les variables quantitatives semblent cohérentes en terme de grandeur suite à différentes suppressions des données atypiques.

In [ ]:
dfDep.drop(["prix m2"],inplace=True,axis=1)

### Les variables catégorielles

In [ ]:
dfDep["nature_mutation"].value_counts().sort_index().plot(kind="bar")
plt.title("Distribution selon les natures de mutation")
plt.xlabel("nature mutation")
plt.ylabel("Count")
plt.show()



La plupart des biens sont des ventes. On va pouvoir supprimer cette variable

In [ ]:
dfDep.drop(['nature_mutation'], inplace=True, axis=1)

In [ ]:
dfDep["type_local"].value_counts().sort_index().plot(kind="bar")
plt.title("Distribution selon le type local")
plt.xlabel("type local")
plt.ylabel("Count")
plt.show()

On peut se poser la question de la pertinence de cette variable car on a essentiellement deux modalités qui jouent

In [ ]:
dfDep["nombre_pieces_principales"].value_counts()

Ce champs apparait mal instancié avec des valeurs abberantes

In [ ]:
dfDep.drop(['nombre_pieces_principales'], inplace=True, axis=1)

In [ ]:
dfDep["nom_commune"].value_counts()

### Conclusion sur l'analyse univariée

On a pu voir que :
* le jeu de données est constituée d'un an et demi d'historique
* la target a été retravaillé pour éliminer les valeurs aberrantes. Il reste néanmoins des points atypiques sur le boxplot
* on a rationnalisé certaines variables en enlevant des valeurs aberrantes ou en les éliminant de l'analyse pour certaines varaibles catégorielles.
* on a introduit du feature ingeenering sur les dates

## Analyse bivariée

L'analyse bivariée va consister à regarder l'influence de différentes variables sur la variable cible.

### Les variables catégorielles

In [ ]:
sns.catplot(x="month", y="valeur_fonciere", kind="box", data=dfDep)

Que cela soit en comparant month, year, day, les niveaux semblent relativement identiques mais on a beaucoup de points atypiques

In [ ]:
sns.catplot(x="type_local", y="valeur_fonciere", kind="box", data=dfDep, orient="v")

On peut constater que le prix des maisons est plus élevé que locaux industriels qui sont eux-mêmes à un niveau équivalent par rapport aux appartements en moyenne


In [ ]:
sns.catplot(x="nom_commune", y="valeur_fonciere", kind="box", orient="v",data=dfDep)

Les arrondissements semblent avoir un effet sur les prix

### Les variables quantitatives

In [ ]:
dfDep.info()

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(14,10)
sns.regplot(x="lot1_surface_carrez",y="valeur_fonciere",data=dfDep, ax=ax1)
sns.regplot(x="nombre_lots",y="valeur_fonciere",data=dfDep,ax=ax2)
sns.regplot(x="surface_reelle_bati",y="valeur_fonciere",data=dfDep,ax=ax3)
#sns.regplot(x="Prix m2",y="valeur_fonciere",data=dfDep,ax=ax4)

La valeur foncière augmente avec la surface, le nombre de lots, la surface réelle.

In [ ]:
sns.heatmap(dfDep.corr(), cmap="YlOrRd")
plt.title("Corrélations des variables continues")
plt.show()
dfDep.corr()

La valeur foncière est fortement corrélée à la surface carrez ou réelle, faiblement au nombre de lots et aux coordonnées gps

### Conclusion sur l'analyse bivariée

La target est sensible : 
* à la surface réelle ou carrez
* au prix du m2
* à la commune
Par contre, elle ne semble pas tellement sensible au mois, jour, année.
La difficulté vient surtout du nombre de points atypiques importants.

## Analyse multivariée

In [ ]:
#sns.relplot(x="lot1_surface_carrez", y="surface_reelle_bati", size="valeur_fonciere", sizes=(15, 100), data=dfDep);
#sns.catplot(x="lot1_surface_carrez", y="valeur_fonciere", hue="type_local",kind="bar",data=dfDep);

# Modelisation

## Preprocessing pour scikit-learn¶


In [ ]:
dfDep.info()

### Gestion des données manquantes
Les méthodes numériques d'apprentissage ne gèrent pas les NaN ou null sur les valeurs numériques

In [ ]:
dfDep.isna().sum()

In [ ]:
dfDep.drop(['lot1_surface_carrez'],inplace=True, axis=1)
dfDep.drop(dfDep[dfDep['surface_reelle_bati'].isna()].index, inplace=True, axis=0)

In [ ]:
dfDep.isna().sum()

### Construction des ensembles X et y à partir du dataframe

On peut enlever l'id qui est un champ purement technique ainsi que la date et tous les éléments de type string

In [ ]:
dfDep["id_mutation"].drop_duplicates(inplace=True)
print(dfDep.shape)

In [ ]:
X = dfDep.drop(["valeur_fonciere","id_mutation", "date_mutation", "numero_disposition", "adresse_numero", "adresse_nom_voie","adresse_complete"], axis = 1)
y = dfDep["valeur_fonciere"]
print(f"Shape de X : {X.shape}")
print(f"Shape de y : {y.shape}")
X.head(5)

### Preprocessing sur les variables catégorielles

In [ ]:
categorical_features = X.columns[X.dtypes == "category"].tolist()
print(categorical_features)

Scikit-learn ne reconnait pas les objets de type DataFrame directement, notamment les types catégoriels. Il faut donc préparer nos données afin que les méthodes de scikit-learn puissent les interpréter. Scikit learn requiert un encodage numérique des ces variables. Nous allons donc devoir encoder nos variables explicatives catégorielles à l'aide de variables indicatrices.

In [ ]:
df_dummies =  pd.get_dummies(X[categorical_features], drop_first=True)
X = pd.concat([X.drop(categorical_features, axis=1), df_dummies], axis=1)
X.head(5)

## Train, Test
Nous utilisons scikit-learn pour faire le traitement et étant donné la volumétrie du jeu de données, nous allons prendre 80% pour le train et 20% pour le test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)
print(f"Shape du X_train : {X_train.shape}")
print(f"Shape du y_train : {y_train.shape}")
print(f"Shape du X_test : {X_test.shape}")
print(f"Shape du y_test : {y_test.shape}")

## Preprocessing sur les variables numériques

In [ ]:
numerical_features = dfDep.columns[(dfDep.dtypes == "int64")].tolist() + dfDep.columns[(dfDep.dtypes == "float64")].tolist()
print(numerical_features)

Certaines méthodes d'apprentissage sont sensibles aux problèmes d'échelle sur les valeurs numériques. En preprocessing, on standardise les variables numériques en retranchant leur moyenne et en divisant par l'écart type via Scikit-learn. On réalise ce traitement sur l'ensemble d'apprentissage et on applique cette standardisation sur l'ensemble de test.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Un modèle simple : la régression linéaire

Un premier modèle qui nous servira de *baseline*.

Nous allons aussi introduire l'instanciation sur les données *train*, et nous validerons **ENSUITE** sur les données *test*.

### Modèle de regression sur Train/Test
$$y =\sum_{i=1}^{n} a_i \times x_i + b$$

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

reg.fit(X_train_scaled, y_train)
y_trainPred = reg.predict(X_train_scaled)
y_testPred = reg.predict(X_test_scaled)
print(f"Score sur le train : {reg.score(X_train_scaled,y_train)}")
print(f"Score sur le test : {reg.score(X_test_scaled,y_test)}")

La régression linéaire donne des résultats et il n'y a pas de phénomène de sur-apprentissage.

## Coefficients de la régression linéaire

Un des avantages de la régression linéaire est que nous pouvons obtenir les coefficients associés à chacune des variables. Nous pouvons voir les coefficients qui ont un impact sur le nombre de vélos loués.

Regardons ces coefficients :

In [ ]:
coefficients = pd.Series(reg.coef_.flatten(), index=X.columns).sort_values(ascending=False)
coefficients

In [ ]:
print(f"ordonnee à l'origine : {reg.intercept_}")

In [ ]:
coefficients[np.abs(coefficients)>10000].plot(kind="bar")
plt.title("Regression lineaire coefficient")
plt.ylabel("Coefficient value")
plt.show()

On retrouve des éléments de l'exploration. Certaines communes tirent le prix vers le base comme le 18ème, 19ème au contraire du 6ème, ... L'élément le plus prépondérant est la surface réellement bati. La lattitude et la longitude s'opposent en termes d'effet.

### Evaluation de la régression avec différentes métriques

Nous allons regarder quelques métriques associées aux problématiques de régression :
* L'erreur maximum entre la prédiction et la réalité
* La moyenne des erreurs absolus entre la prédiction et la réalité
* La moyenne des erreurs au carré entre la prédiction et la réalité (MSE)
* Le score R2 qui est le coefficient de détermination en comparant MSE et la variance. Fonction renvoyée par la méthode score de Scikit Learn

In [ ]:
from sklearn import metrics


def regression_metrics(y, y_pred):
    return pd.DataFrame(
        {
            "max_error": metrics.max_error(y_true=y, y_pred=y_pred),
            "mean_absolute_error": metrics.mean_absolute_error(y_true=y, y_pred=y_pred),
            "mean_squared_error": metrics.mean_squared_error(y_true=y, y_pred=y_pred),
            "r2_score": metrics.r2_score(y_true=y, y_pred=y_pred)
        },
        index=[0])

In [ ]:
print("Regression metrics for train data")
print(regression_metrics(y_train, y_trainPred))
print("Regression metrics for test data")
print(regression_metrics(y_test, y_testPred))

Le modèle de regression linéaire n'est pas très bon quelque soit la métrique retenue.

## Arbre de décision et visions ensemblistes
### Arbre de décision


In [ ]:
from sklearn.tree import DecisionTreeRegressor
decisionTree = DecisionTreeRegressor()
decisionTree.fit(X_train_scaled, y_train)
y_trainPred = decisionTree.predict(X_train_scaled)
y_testPred = decisionTree.predict(X_test_scaled)
print(f"Score sur le train de l'arbre de décision : {decisionTree.score(X_train_scaled,y_train)}")
print(f"Score sur le test de l'arbre de décision : {decisionTree.score(X_test_scaled,y_test)}")


In [ ]:
print("Regression metrics with Decision Tree for train data")
print(regression_metrics(y_train, y_trainPred))
print("Regression metrics with Decision Tree for test data")
print(regression_metrics(y_test, y_testPred))

On est dans un cas de surapprentissage puisque l'arbre de décision "fit"  à l'ensemble de train mais ne se généralise pas bien sur l'ensemble de test. Néanmoins la performance est moins bonne que la régression linéaire

In [ ]:
print("Feature importances : \n{}".format(decisionTree.feature_importances_))

In [ ]:
def plot_feature_importances(model):
    n_features = X.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align = 'center')
    plt.yticks(np.arange(n_features), X.columns)
    plt.xlabel("Feature Importance")
    plt.ylabel("Feature")
    plt.ylim(-1,n_features)

In [ ]:
plot_feature_importances(decisionTree)

In [ ]:
featuresImportance = pd.Series(decisionTree.feature_importances_.flatten(), index=X.columns).sort_values(ascending=False)
featuresImportance[(featuresImportance)>0.03].plot(kind="bar")
plt.title("Feature")
plt.ylabel("Feature Importance")
plt.show()

On retrouve la surface réelle et la position géographique du bien.

In [ ]:
for depth in range(5,20):
    decisionTreeMaxDepth = DecisionTreeRegressor(max_depth=depth)
    decisionTreeMaxDepth.fit(X_train_scaled, y_train)
    print(f"Max depth : {depth}")
    print(f"Score sur le train de l'arbre de décision : {decisionTreeMaxDepth.score(X_train_scaled,y_train)}")
    print(f"Score sur le test de l'arbre de décision : {decisionTreeMaxDepth.score(X_test_scaled,y_test)}")

On observe assez vite le surapprentissage lorsqu'on augmente la profondeur de l'arbre

Avantages : 
* On peut contrôler la complexité de l'arbre en jouant sur des paramètres avec la profondeur ou des stratégies d'élagage
* Interprétabilité des décisions
* Pas de problématique de prise en compte des échelles différentes entre les variables (même si dans notre cas, nous travaillons sur des données standardisées)

Inconvénient majeur :
* Même en jouant sur la complexité de l'arbre, un arbre tend au surapprentissage et fournit de piètre performance de généralisation

### Random Forest


In [ ]:
from sklearn.ensemble import RandomForestRegressor
nbTree = 100
print(f"Nombre d'arbres considérés : {nbTree}")
for depth in [5,10,15,20,30, 40]:
    randomForest = RandomForestRegressor(n_estimators=nbTree, random_state=2, max_depth=depth)
    randomForest.fit(X_train_scaled, y_train)
    print(f"--- Max depth : {depth}")
    print(f"---------Score sur le train avec RandomForest : {randomForest.score(X_train_scaled,y_train)}")
    print(f"---------Score sur le test avec RandomForest : {randomForest.score(X_test_scaled,y_test)}")

On observe avec Random Forest une amélioration du score fonction de la profondeur considérées avec un surapprentissage de plus en plus important. 

### GridSearch et Validation croisée

Nous allons creuser un peu plus loin afin d'améliorer RandomForest en optimisant les hyperparamètres du modèle. Pour ce faire nous allons procéder par validation croisée avec 5 plis sur l'ensemble d'apprentissage. 
A l'aide de celle-ci, nous allons chercher quel(s) paramètre(s) nous donne(nt) le meilleur score et enfin nous évaluerons la qualité du modèle sur le jeu de données test.

Les paramètres que nous allons chercher à optimiser dans RandomForest sont :
* le paramètre max_depth qui correspond à la profondeur de l'arbre 
* le nombre d'arbres à considérer dans la forêt
* le nombre de features maximale à considérer

In [ ]:
from sklearn.model_selection import GridSearchCV
# grille de valeurs
params = [{"max_depth": [10,15,20], "n_estimators": [100,200,300,500], "max_features": [12, 15, 20, 25]}]

gridSearchCV = GridSearchCV(
    RandomForestRegressor(),
    params,
    cv=5,
    n_jobs=-1,
    return_train_score=True)
gridSearchCV.fit(X_train_scaled, y_train)

In [ ]:
print("Score sur le test : {:.2f}".format(gridSearchCV.score(X_test_scaled,y_test)))

In [ ]:
print("Best parameters : {}".format(gridSearchCV.best_params_))
print("Best cross-validation score : {:.2f}".format(gridSearchCV.best_score_))

In [ ]:
print("Best estimator:\n{}".format(gridSearchCV.best_estimator_))

In [ ]:
featuresImportance = pd.Series(gridSearchCV.best_estimator_.feature_importances_.flatten(), index=X.columns).sort_values(ascending=False)
featuresImportance[(featuresImportance)>0.03].plot(kind="bar")
plt.title("Feature")
plt.ylabel("Feature Importance")
plt.show()

A travers une validation croisée et un grid search, on obtient un paramétrage via Random Forest et on peut visualiser les variables qui ont de l'importance. On retrouve des variables explicatives en lien avec notre analyse exploratoire. On est aussi dans un cas où il n'y a pas de surapprentissage. 

In [ ]:
y_testPred = gridSearchCV.best_estimator_.predict(X_test_scaled)
print("Regression metrics pour la forêt aléatoire optimisée for test data")
print(regression_metrics(y_test, y_testPred))

In [ ]:
y_trainPred = gridSearchCV.best_estimator_.predict(X_train_scaled)
print("Regression metrics pour la forêt aléatoire optimisée for train data")
print(regression_metrics(y_train, y_trainPred))

## Sauvegarde du modèle 

In [ ]:
#from joblib import dump, load

In [ ]:
#dump(gridSearchCV.best_estimator_.predict, 'sauvegardeModele.joblib') 

In [ ]:
#clf = load('sauvegardeModele.joblib') 

In [ ]:
#clf.predict(X_test_scaled)


# Conclusion

## Sur le travail réalisé
* L'analyse univariée et multivariée ont permis de mettre en évidence des liens entre les variables explicatives et à expliquer
* Le featuring Ingeenering a été un travail réalisé sur les dates pour essayer de voir les liens avec la variable à prédire.
* Les modèles linéaires donne des résultats pas très intéressants sur certaines métriques
* Un modèle basé sur des arbres de décision permet d'obtenir des meilleurs résultats par rapport à la regression linéaire. Une optimisation des paramètres a pu être mise en oeuvre via validation croisée et grille de recherche

## Sur les perspectives
* Sur le code : la mise en place de Pipe avec l'utilisation de OneHotEncoder et StandardScaler.
* Sur les modèles : tester d'autres modèles pour améliorer la prévision. On peut penser à une régression polynomiale ou boosting d'arbres de régression, ou des modèles traitant spécifiquement de séries temporelles.
* Un traitement des points atypiques a été réalisé avec aussi de l'imputation mais il faudrait passer plus de temps sur la compréhension des données.